赛题网址
http://www.pkbigdata.com/common/cmpt/%E7%94%A8%E6%88%B7%E8%B4%B7%E6%AC%BE%E9%A3%8E%E9%99%A9%E9%A2%84%E6%B5%8B_%E8%B5%9B%E4%BD%93%E4%B8%8E%E6%95%B0%E6%8D%AE.html

In [112]:
bank_record_count = bank_detail.groupby(by="userID",as_index=False).size()
bank_record_count.name = "bank_detail_record_count"
bank_record_count = bank_record_count.reset_index()
bank_record_count.head()


There are 10003 customers has bank_detail info.


In [114]:
bank_income_stats = bank_detail.loc[bank_detail["transaction_type"]==0,
                        ("userID","transaction_amount")].groupby(
    by="userID",as_index=False).aggregate([np.sum, np.size])
bank_income_stats.columns = ["bank_income_sum","bank_income_count"]
bank_income_stats = bank_income_stats.reset_index()

bank_outcome_stats = bank_detail.loc[bank_detail["transaction_type"]==1,
        ("userID","transaction_amount")].groupby(
    by="userID",as_index=False).aggregate([np.sum, np.size])
bank_outcome_stats.columns = ["bank_outcome_sum","bank_outcome_count"]
bank_outcome_stats = bank_outcome_stats.reset_index()

bank_salaryincome_stats = bank_detail.loc[
    (bank_detail["transaction_type"]==0)&(bank_detail["salary_income"]==1),
    ("userID","transaction_amount")].groupby(by="userID",as_index=False
                                            ).aggregate([np.sum, np.mean])
bank_salaryincome_stats.columns = ["bank_salaryincome_sum",
                                   "bank_salaryincome_mean"]
bank_salaryincome_stats = bank_salaryincome_stats.reset_index()

shape:   (10002, 3)


,userID,bank_income_sum,bank_income_count
0,1,480.692762,35.0
1,3,2278.873446,172.0
2,4,1164.342384,96.0
3,9,186.833251,13.0
4,10,1793.642133,141.0


In [ ]:
bank_detail_stats.loc[bank_detail_stats["bank_salaryincome_sum"].notnull(), 
                      "bank_salary_record"] = 2 
bank_detail_stats.loc[bank_detail_stats["bank_salaryincome_sum"].isnull(), 
                      "bank_salary_record"] = 1
bank_detail_stats.drop(["bank_salaryincome_sum","bank_salaryincome_mean"],
                       axis=1, inplace=True)

bank_detail_stats["bank_out/income"] = bank_detail_stats[
    "bank_outcome_sum"]/bank_detail_stats["bank_income_sum"]

bank_detail_stats.fillna(0, inplace=True)  


### 4.1.2 描述性统计  
&ensp;&ensp;  
1. 将bank_detail_stats表里的训练集部分提取出来，将之和overdue_train表依照userID链接起来，然后，根据某个userID是否违约，提取出违约部分和正常部分。
2. 对正常部分的“支出占收入比”字段做直方图，同时，对违约部分的“支出占收入比”也做直方图，两个直方图在一张图上，观察违约人群与正常人群在此字段上的分布差异。

&ensp;&ensp;前两幅图是训练集里有bank信息的客户中，在“bank_out/income”字段上是 正常 和 违约 两种情况的分布。（注意，直方图里并没有叠加，而是正常的分布 与 违约的分布 画在了一起）。可见，大部分用户，不论正常/违约，其“支出收入比值”都在0~6.5之间，即支出是收入的0~6.5倍。但是，有个别客户，其支出收入比到了更高的10~20倍，这些我认为太异常了，建议删去。bank_detail_stats_train_Y_0中，大于6.5的有60个，bank_detail_stats_train_Y_1中，大于6.5的有11个。所以一共有 71 个异常的userID，存储在bank_train_outincome_ratio_outlier里。

## 4.2 browse_history 整合

&ensp;&ensp;browse_behavior信息以后要整合到总表里，每个用户一条记录。而有browse_behavior信息的用户有85.37% 比较多。
1. 以browse_time与loan_time的对比，对数据进行分组。检查发现browse_time里没有值0（表示时间未知），所以，根据 放款前browse_time<=loantime 和 放款后 browse_time>loantime 对数据集进行分割。后发现，（训练集+未知数据集共约7万人），有5万6千人有放款前数据，这是可以接受的。但是只有1万5千人有放款后数据，太少了，预计使用放款后数据会产生很多缺失值。所以决定：仅仅使用放款前数据 与 不分放款前放款后的总体数据。
2. 按时间分割得到数据后，对每个用户统计其所具有的浏览子行为种类数、每种编号的浏览子行为对应的浏览行为数据的sum,max,mean,median,min.
3. 因为以上操作都是针对每个用户本身进行的，所以可以训练集和未知数据集一起做。

### 4.2.1 groupby and feature creation 
1. 统计全部时间（不论放款前后）的信息 和 放款前的信息，两表结合起来。
2. 创建了新特征，即每个用户在所有时间，对部分种类的浏览行为的浏览数据的sum,max,mean,median,min,count。以及每个用户在放款前的时间里，对部分种类的浏览行为的浏览数据的sum,max,mean,median,min,count。
3. 两表内部的缺失值用 0 填充，但两表结合后得到的 browse_history_stats 表的缺失值，留待以后算法填充。

In [133]:
browse_history.loc[browse_history["browse_time"]==0,
                   ["userID","browse_time"]]

,userID,browse_time


In [134]:
# 先把 browse_history和 loan_time 依照 userID 融合在一起，得到 browse_history_L，L代表此表有 loantime列。
# 将 browsetime<=loantime的记录选取出来，即放款前的所有记录，browse_history_L_before ，before代表放款前。


browse_history_L = pd.merge(browse_history,loan_time, on="userID",
                            how="left")
browse_history_L_before = browse_history_L.loc[
    browse_history_L['browse_time']<=browse_history_L['loan_time'],:]
browse_history_L_after = browse_history_L.loc[
     browse_history_L['browse_time']>browse_history_L['loan_time'],:]
tmp49 = len(browse_history_L["userID"].unique())
tmp51 = len(browse_history_L_before["userID"].unique())
tmp52 = len(browse_history_L_after["userID"].unique())
tmp53 = len(user_info["userID"])
print("user number with browse_history_L:  ",tmp49)
print("user number with browse_history_L_before:  ",tmp51)
print("user number with browse_history_L_after:  ",tmp52)
print("bill percent:",tmp49/tmp53)
print("before percent:",tmp51/tmp53)
print("after percent:",tmp52/tmp53)

# 训练集+未知数据集约 7万人，其中有 5万9千人，即85%有browse数据。
#有81%有放款前数据，但仅有23%有放款后数据。

user number with browse_history_L:   59327
user number with browse_history_L_before:   56091
user number with browse_history_L_after:   15990
bill percent: 0.8536873156342183
before percent: 0.8071228145909778
after percent: 0.23008849557522124


In [135]:
browse_history_L_before.head(5)

,userID,browse_time,browse_behavior,browse_num,loan_time
0,2,5893398099,143,3,5914855887
1,2,5912610799,189,1,5914855887
2,2,5912610799,44,5,5914855887
3,2,5907360322,45,1,5914855887
4,2,5907360322,118,1,5914855887


In [136]:
# 统计放款前，每个用户拥有的 browse_history条目记录数，并把它起名为 browse_before_recordcount
# 由 shape 可知，共有 56091 个用户有 放款前 browse_history 信息。此表无缺失。

gb1 = browse_history_L_before.groupby(by="userID")
browse_before_recordcount = gb1.size()
browse_before_recordcount.name = "browse_before_recordcount"
browse_before_recordcount = pd.DataFrame(browse_before_recordcount)

shape:   (56091, 1)


,browse_before_recordcount
userID,
2,1305
3,342
4,364
6,15
7,1008


,browse_before_recordcount
count,56091.000000
mean,465.893994
std,708.549718
min,1.000000
25%,126.000000
50%,272.000000
75%,552.000000
max,33972.000000


In [138]:
browse_before_kindcount = gb1["browse_num"].apply(
    lambda x:np.size(np.unique(x)))
browse_before_kindcount.name = 'browse_before_kindcount'
browse_before_kindcount = pd.DataFrame(browse_before_kindcount)

shape:   (56091, 1)


,browse_before_kindcount
userID,
2,8
3,6
4,8
6,1
7,7


In [139]:
gb2 = browse_history_L_before.groupby(by=['userID','browse_num'])
gb3 = gb2['browse_behavior'].aggregate(
    [np.sum,np.max,np.mean,np.median,np.min,np.size])
gb4 = gb3.unstack(level = -1)

In [141]:
L1 = ['sum','max','mean','median','min','count']
L2 = ['1','2','3','4','5','6','7','8','9','10','11']
L = ['_'.join(['browse_BLbn',j,i]) for i in L1 for j in L2]    
gb4.columns = L  
browse_before_behavior = gb4

shape:   (56091, 66)


,browse_BLbn_1_sum,browse_BLbn_2_sum,browse_BLbn_3_sum,browse_BLbn_4_sum,browse_BLbn_5_sum,browse_BLbn_6_sum,browse_BLbn_7_sum,browse_BLbn_8_sum,browse_BLbn_9_sum,browse_BLbn_10_sum,...,browse_BLbn_2_count,browse_BLbn_3_count,browse_BLbn_4_count,browse_BLbn_5_count,browse_BLbn_6_count,browse_BLbn_7_count,browse_BLbn_8_count,browse_BLbn_9_count,browse_BLbn_10_count,browse_BLbn_11_count
userID,,,,,,,,,,,,,,,,,,,,,
2,59820.0,NaN,11160.0,22725.0,19230.0,4710.0,6150.0,13455.0,NaN,14250.0,...,NaN,90.0,150.0,105.0,135.0,105.0,105.0,NaN,75.0,NaN
3,19968.0,NaN,NaN,7272.0,2694.0,1200.0,2892.0,NaN,NaN,4560.0,...,NaN,NaN,48.0,30.0,24.0,42.0,NaN,NaN,24.0,NaN
4,17824.0,NaN,572.0,9696.0,2188.0,1600.0,1656.0,668.0,NaN,4560.0,...,NaN,4.0,64.0,36.0,32.0,36.0,4.0,NaN,24.0,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,1650.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN
7,53298.0,NaN,NaN,21000.0,10346.0,5740.0,7280.0,2338.0,NaN,12418.0,...,NaN,NaN,140.0,98.0,98.0,112.0,14.0,NaN,84.0,NaN


In [ ]:
browse_before_stats.drop(tmp16["column_name"].tolist(), 
                         axis=1, inplace = True)

browse_before_stats.fillna(0, inplace=True)

In [147]:
# 同理，这次总结 所有时间（放款前和放款后）的同样的字段。

# browse_history和 loan_time 依照 userID 融合在一起，
得到 browse_history_L（之前已得到），L代表此表有 loantime列。
# browse_history_L 

# 统计所有时间，每个用户拥有的 browse_history条目记录数，并把它起名为 browse_all_recordcount, all 代表所有时间
# 由 shape 可知，共有 59327 个用户有 放款前 browse_history 信息。此表无缺失。
gb5 = browse_history_L.groupby(by="userID")
browse_all_recordcount = gb5.size()
browse_all_recordcount.name = "browse_all_recordcount"
browse_all_recordcount = pd.DataFrame(browse_all_recordcount)
print("shape:  ",browse_all_recordcount.shape)

browse_all_kindcount = gb5["browse_num"].apply(lambda x:np.size(np.unique(x)))
browse_all_kindcount.name = 'browse_all_kindcount'
browse_all_kindcount = pd.DataFrame(browse_all_kindcount)
print("shape:  ",browse_all_kindcount.shape)

gb6 = browse_history_L.groupby(by=['userID','browse_num'])
gb7 = gb6['browse_behavior'].aggregate([np.sum,np.max,np.mean,np.median,np.min,np.size])
gb8 = gb7.unstack(level = -1)

L1 = ['sum','max','mean','median','min','count']
L2 = ['1','2','3','4','5','6','7','8','9','10','11']
L = ['_'.join(['browse_ALLbn',j,i]) for i in L1 for j in L2]    
gb8.columns = L  # 改列名，同时将原来的双重列索引 双改成单层列索引。
browse_all_behavior = gb8    # all代表 所有时间。
print("shape:  ",browse_all_behavior.shape)

# 将 3 张表依 userID索引 join  起来，得到 browse_all_stats表。
tmp17 = browse_all_recordcount.join(browse_all_kindcount)
browse_all_stats = tmp17.join(browse_all_behavior)
print("shape:  ", browse_all_stats.shape)

shape:   (59327, 1)
shape:   (59327, 1)
shape:   (59327, 66)
shape:   (59327, 68)
shape:   (24, 3)


In [ ]:
tmp20 = tmp18.loc[tmp18["nanpercent"]>0.49]
browse_all_stats.drop(tmp20["column_name"].tolist(), axis=1, inplace = True)

browse_all_stats.fillna(0, inplace=True) 

browse_history_stats = browse_all_stats.join(browse_before_stats
                                            ).reset_index()

### 4.2.2 描述性统计

1. 首先对 训练集里有browse信息的用户，按违约与正常分开，看各自的 browse_all_recordcount 字段，即每个用户的浏览记录条目数。用箱线图和直方图展示，观察条目数在不同类别中的分布情况，并查看是否有异常值。
2. 对 训练集里有browse信息的用户，按违约与正常分开，看各自的第1,4,5,6,7,10号浏览行为的平均值，如 browse_ALLbn_5_mean 字段，即每个用户在第5种浏览行为上的浏览数据平均值。用箱线图和直方图展示，观察均值在不同类别中的分布情况，并查看是否有异常值。

&ensp;&ensp; 由箱线图和直方图可见，训练集里有 浏览信息的用户里，每个用户收集到的浏览信息记录数的分布非常有偏。有个别用户的浏览信息records数特别多，我认为属于异常点，需要删去。browse_history_stats_train_Y里，以5000为分割点查找，5000以上的人有163个，6000以上103个，7000以上67个，8000以上39个。我选择 8000这个分割点，建议将8000以上的训练集用户删去。  
&ensp;&ensp; 从分布情况来说，不论违约还是正常，浏览信息条目数的分布都是有偏的。

The number of outliers in all_recordcount is: 39


## 4.3 bill_detail 整合

1. 有96.15%的用户有bill_detail信息，比较多。统计 bill_detail 表中，账单时间早于和晚于 loantime的条目数的比例。经统计，决定对放款前、放款后、所有时间进行计算。
2. 每个用户对应多条记录，先不考虑银行ID问题，对一个用户的所有条目，在数值字段上进行sum,max,median,mean,min统计。
3. 假设每个用户在一个银行只有一张银行卡，寻找每个银行卡的信用额度，相加得到一个用户的总信用额度。对每个用户的每张信用卡的若干字段取平均值。然后再对每个用户的各个信用卡的均值，在数值字段上进行sum,max,median,min统计。
4. 创建特征，如每个用户的bill_detail条目数、账期逾期概率、超额消费率、溢缴率，这是放款前、放款后、所有时间 三张表都有的。
5. 创建特征，每个用户的银行卡数目、每个用户的银行卡额度。这是单独的，不属于放款前、放款后、所有时间三张表。
6. 将三张表+2个独立特征 链接，成为 bill_detail_stats。

### 4.3.1 groupby and feature creation

1. 分放款前 和 所有时间（不论放款前后，时间已知未知）。
2. 对于放款前行条目，删掉 “银行ID” 这列，对每个用户ID在所有银行办理的全部银行卡 进行统计。

In [162]:
bill_detail_L = pd.merge(bill_detail,loan_time, on="userID", how="left")
bill_detail_L_unknown = bill_detail_L.loc[bill_detail_L['bill_time']==0,:]
bill_detail_L_before = bill_detail_L.loc[
    (bill_detail_L['bill_time']<=bill_detail_L['loan_time'])&(
        bill_detail_L['bill_time']!=0),:]
bill_detail_L_after = bill_detail_L.loc[
    bill_detail_L['bill_time']>bill_detail_L['loan_time'],:]
tmp44 = len(bill_detail_L["userID"].unique())
tmp45 = len(bill_detail_L_unknown["userID"].unique())   
tmp46 = len(bill_detail_L_before["userID"].unique())
tmp47 = len(bill_detail_L_after["userID"].unique())
tmp48 = len(user_info["userID"])


user number with bill_detail_L:   66584
user number with bill_detail_L_unknown:   17498
user number with bill_detail_L_before:   54614
user number with bill_detail_L_after:   42758
bill percent: 0.9581120943952802
unknown percent: 0.2517878984099576
before percent: 0.7858694870134542
after percent: 0.615267285416217


In [164]:
# 统计放款前，每个用户拥有的 bill_detail 条目记录数，并把它起名为 bill_before_recordcount
# 由 shape 可知，共有 54614 个用户有 放款前 bill_detail 信息。
# 此表无缺失。

gb9 = bill_detail_L_before.groupby(by="userID")
bill_before_recordcount = gb9.size()
bill_before_recordcount.name = "bill_before_recordcount"
bill_before_recordcount = pd.DataFrame(bill_before_recordcount)
print("shape:  ",bill_before_recordcount.shape)
bill_before_recordcount.head()

shape:   (54614, 1)


,bill_before_recordcount
userID,
2,7
4,20
6,16
7,19
9,12


In [165]:
bill_detail_L_before_pick = bill_detail_L_before.drop(
    ["bill_time","bankID","loan_time","repay_status"],axis = 1)
gb10 = bill_detail_L_before_pick.groupby(by="userID")
bill_detail_L_before_pick_stats = gb10.aggregate(
    [np.sum,np.max,np.median,np.mean,np.min])
print("shape:  ",bill_detail_L_before_pick_stats.shape)
bill_detail_L_before_pick_stats.head()

shape:   (54614, 55)


last_bill_amount                                              \
                    sum       amax     median       mean       amin   
userID                                                                
2             96.598251  21.580308  17.389606  13.799750   0.000000   
4            144.813292  19.483781   0.000000   7.240665   0.000000   
6            314.927082  20.681443  19.638586  19.682943  18.654047   
7            267.402239  19.861357  19.143810  14.073802   0.000000   
9            241.771564  21.330732  20.248808  20.147630  18.587637   

       last_repay_amount                                             ...   \
                     sum       amax     median       mean       amin ...    
userID                                                               ...    
2              97.419017  21.600903  17.402921  13.917002   0.000000 ...    
4             105.148045  19.483836   0.000000   5.257402   0.000000 ...    
6             270.925010  21.177840  18.997091  16.932813   0.000000 ...    
7             342.363130  21.135796  20.664428  18.019112   0.000000 ...    
9             243.389228  21.622615  20.299273  20.282436  18.907778 ...    

       available_balance                                        borrow_cash  \
                     sum       amax median      mean       amin         sum   
userID                                                                        
2               0.000000   0.000000    0.0  0.000000   0.000000   20.376736   
4              36.516325  19.662141    0.0  1.825816   0.000000  168.500217   
6              62.908887  20.464738    0.0  3.931805 -18.105068  218.464023   
7              80.907026  20.903642    0.0  4.258265   0.000000  270.987654   
9               0.000000   0.000000    0.0  0.000000   0.000000  138.242700   

                                              
             amax     median       mean amin  
userID                                        
2       20.376736   0.000000   2.910962  0.0  
4       18.767298   0.000000   8.425011  0.0  
6       20.559057  19.278123  13.654001  0.0  
7       19.460445  19.278123  14.262508  0.0  
9       20.376736  19.278123  11.520225  0.0  

[5 rows x 55 columns]

In [167]:
L1 = ['sum','max',"median",'mean','min']
L2 = ['last_bill_amount','last_repay_amount','credit_limit','current_bill_balance','current_bill_minrepay','consume_num','current_bill_amount','adjust_amount','cyclic_interest','available_balance','borrow_cash']
L = ['_'.join(['bill_BL',i,j]) for i in L2 for j in L1]    

bill_detail_L_before_pick_stats.columns = L
print("shape:  ",bill_detail_L_before_pick_stats.shape)
bill_detail_L_before_pick_stats.head()

shape:   (54614, 55)


,bill_BL_last_bill_amount_sum,bill_BL_last_bill_amount_max,bill_BL_last_bill_amount_median,bill_BL_last_bill_amount_mean,bill_BL_last_bill_amount_min,bill_BL_last_repay_amount_sum,bill_BL_last_repay_amount_max,bill_BL_last_repay_amount_median,bill_BL_last_repay_amount_mean,bill_BL_last_repay_amount_min,...,bill_BL_available_balance_sum,bill_BL_available_balance_max,bill_BL_available_balance_median,bill_BL_available_balance_mean,bill_BL_available_balance_min,bill_BL_borrow_cash_sum,bill_BL_borrow_cash_max,bill_BL_borrow_cash_median,bill_BL_borrow_cash_mean,bill_BL_borrow_cash_min
userID,,,,,,,,,,,,,,,,,,,,,
2,96.598251,21.580308,17.389606,13.799750,0.000000,97.419017,21.600903,17.402921,13.917002,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,20.376736,20.376736,0.000000,2.910962,0.0
4,144.813292,19.483781,0.000000,7.240665,0.000000,105.148045,19.483836,0.000000,5.257402,0.000000,...,36.516325,19.662141,0.0,1.825816,0.000000,168.500217,18.767298,0.000000,8.425011,0.0
6,314.927082,20.681443,19.638586,19.682943,18.654047,270.925010,21.177840,18.997091,16.932813,0.000000,...,62.908887,20.464738,0.0,3.931805,-18.105068,218.464023,20.559057,19.278123,13.654001,0.0
7,267.402239,19.861357,19.143810,14.073802,0.000000,342.363130,21.135796,20.664428,18.019112,0.000000,...,80.907026,20.903642,0.0,4.258265,0.000000,270.987654,19.460445,19.278123,14.262508,0.0
9,241.771564,21.330732,20.248808,20.147630,18.587637,243.389228,21.622615,20.299273,20.282436,18.907778,...,0.000000,0.000000,0.0,0.000000,0.000000,138.242700,20.376736,19.278123,11.520225,0.0


In [168]:
tmp33 = bill_detail_L_before[['userID','repay_status']].groupby(by = 'userID').aggregate([np.sum, np.size])
tmp33.columns = ['tmp_sum','tmp_count'] 
tmp33 = tmp33.assign(bill_BL_overdue_percent = tmp33['tmp_sum']/tmp33['tmp_count']) # assign函数为 tmp33增加一列。
bill_BL_overdue_percent = pd.DataFrame(tmp33['bill_BL_overdue_percent'])
print("shape:  ",bill_BL_overdue_percent.shape)
bill_BL_overdue_percent.head()

shape:   (54614, 1)


,bill_BL_overdue_percent
userID,
2,0.0
4,0.0
6,0.0
7,0.0
9,0.0


In [169]:
CF.Calnan(bill_BL_overdue_percent)

,column_name,nancount,nanpercent
0,bill_BL_overdue_percent,0,0.0


In [170]:
# 可用余额 <0 表示超额消费，用每个用户放款前 可用余额<0 的次数 比上 此用户总的记录数，得到 放款前的超额消费率。
# bill_BL_excess_percent 表示（训练集+未知数据集里）有放款前bill_detail信息的用户在放款前的 超额消费率。
# 本表无缺失。

tmp35 = bill_detail_L_before[['userID','available_balance']]
tmp35["excess"] = tmp35['available_balance']<0
tmp36 = tmp35.groupby("userID").aggregate({"available_balance":np.size, "excess":np.sum})
tmp36.columns = ["num","excess_num"]
tmp36 = tmp36.assign(bill_BL_excess_percent = tmp36["excess_num"]/tmp36["num"])
bill_BL_excess_percent = pd.DataFrame(tmp36["bill_BL_excess_percent"])
print("shape:  ",bill_BL_excess_percent.shape)
bill_BL_excess_percent.head()

C:\Users\shuangjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


shape:   (54614, 1)


,bill_BL_excess_percent
userID,
2,0.0000
4,0.0000
6,0.0625
7,0.0000
9,0.0000


In [171]:
CF.Calnan(bill_BL_excess_percent)

,column_name,nancount,nanpercent
0,bill_BL_excess_percent,0,0.0


In [172]:
# 本期账单余额 <0 表示溢缴，用每个用户放款前 本期账单余额 <0  的次数 比上 此用户总的记录数，得到 放款前的溢缴率。
# bill_BL_overpay_percent 表示（训练集+未知数据集里）有放款前bill_detail信息的用户在放款前的 溢缴率。
# 本表无缺失。

tmp42 = bill_detail_L_before[['userID','current_bill_balance']]
tmp42["overpay"] = tmp42['current_bill_balance']<0
tmp43 = tmp42.groupby("userID").aggregate({"current_bill_balance":np.size, "overpay":np.sum})
tmp43.columns = ["num","overpay_num"]
tmp43 = tmp43.assign(bill_BL_overpay_percent = tmp43["overpay_num"]/tmp43["num"])
bill_BL_overpay_percent = pd.DataFrame(tmp43["bill_BL_overpay_percent"])
print("shape:  ",bill_BL_overpay_percent.shape)
bill_BL_overpay_percent.head()

C:\Users\shuangjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


shape:   (54614, 1)


,bill_BL_overpay_percent
userID,
2,0.000000
4,0.000000
6,0.062500
7,0.315789
9,0.083333


In [173]:
CF.Calnan(bill_BL_overpay_percent)

,column_name,nancount,nanpercent
0,bill_BL_overpay_percent,0,0.0


In [174]:
tmp39 = bill_detail_L_before[[ 'userID', 'bankID','last_bill_amount','last_repay_amount','current_bill_balance',
       'current_bill_minrepay', 'consume_num', 'current_bill_amount',
       'adjust_amount', 'cyclic_interest', 'available_balance', 'borrow_cash']]
tmp40 = tmp39.groupby(['userID', 'bankID']).aggregate(np.mean)
tmp40.columns = ['bill_BL_cardM_'+i for i in tmp40.columns]  # 改列名。BL代表放款前，cardM代表 每张卡的平均值
bill_BL_cardM = tmp40
tmp41 = bill_BL_cardM.reset_index(level=-1).drop("bankID",axis=1).groupby("userID")
bill_BL_cardM_stats = tmp41.aggregate([np.sum,np.max,np.median,np.min])

L1 = ['sum','max',"median",'min']  # 改列名
L2 = ['last_bill_amount','last_repay_amount','current_bill_balance',
       'current_bill_minrepay', 'consume_num', 'current_bill_amount',
       'adjust_amount', 'cyclic_interest', 'available_balance', 'borrow_cash']
L = ['_'.join(['bill_BL_cardM_',i,j]) for i in L2 for j in L1]    
bill_BL_cardM_stats.columns = L
print("shape:  ",bill_BL_cardM_stats.shape)
bill_BL_cardM_stats.head()

shape:   (54614, 40)


,bill_BL_cardM__last_bill_amount_sum,bill_BL_cardM__last_bill_amount_max,bill_BL_cardM__last_bill_amount_median,bill_BL_cardM__last_bill_amount_min,bill_BL_cardM__last_repay_amount_sum,bill_BL_cardM__last_repay_amount_max,bill_BL_cardM__last_repay_amount_median,bill_BL_cardM__last_repay_amount_min,bill_BL_cardM__current_bill_balance_sum,bill_BL_cardM__current_bill_balance_max,...,bill_BL_cardM__cyclic_interest_median,bill_BL_cardM__cyclic_interest_min,bill_BL_cardM__available_balance_sum,bill_BL_cardM__available_balance_max,bill_BL_cardM__available_balance_median,bill_BL_cardM__available_balance_min,bill_BL_cardM__borrow_cash_sum,bill_BL_cardM__borrow_cash_max,bill_BL_cardM__borrow_cash_median,bill_BL_cardM__borrow_cash_min
userID,,,,,,,,,,,,,,,,,,,,,
2,27.746164,14.386405,13.873082,13.359759,27.953095,14.393363,13.976548,13.559732,40.247085,21.560070,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,6.792245,6.792245,3.396123,0.0
4,18.101661,18.101661,0.000000,0.000000,13.143506,13.143506,0.000000,0.000000,55.198366,19.540666,...,0.0,0.0,4.564541,2.457768,2.106773,0.0,23.357756,18.767298,4.590458,0.0
6,59.123257,20.342580,19.469878,19.310799,50.966866,18.985339,16.090681,15.890846,51.685534,19.867388,...,0.0,0.0,12.581777,12.581777,0.000000,0.0,39.837180,20.559057,19.278123,0.0
7,42.173386,16.874108,12.649639,0.000000,65.808590,20.756443,16.105886,12.840374,16.373938,19.401172,...,0.0,0.0,20.226757,20.226757,0.000000,0.0,38.738568,19.460445,9.639061,0.0
9,60.638539,21.013929,20.231336,19.393275,61.059657,21.141414,20.292014,19.626228,47.735878,20.305336,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,39.654859,20.376736,19.278123,0.0


In [176]:
tmp54 = bill_before_recordcount.join(bill_detail_L_before_pick_stats)
tmp55 = tmp54.join(bill_BL_overdue_percent)
tmp56 = tmp55.join(bill_BL_excess_percent)
tmp57 = tmp56.join(bill_BL_overpay_percent)
bill_before_stats = tmp57.join(bill_BL_cardM_stats)
print("shape:  ", bill_before_stats.shape)
bill_before_stats.head()

# 这里用 merge 也可以实现需求。

shape:   (54614, 99)


,bill_before_recordcount,bill_BL_last_bill_amount_sum,bill_BL_last_bill_amount_max,bill_BL_last_bill_amount_median,bill_BL_last_bill_amount_mean,bill_BL_last_bill_amount_min,bill_BL_last_repay_amount_sum,bill_BL_last_repay_amount_max,bill_BL_last_repay_amount_median,bill_BL_last_repay_amount_mean,...,bill_BL_cardM__cyclic_interest_median,bill_BL_cardM__cyclic_interest_min,bill_BL_cardM__available_balance_sum,bill_BL_cardM__available_balance_max,bill_BL_cardM__available_balance_median,bill_BL_cardM__available_balance_min,bill_BL_cardM__borrow_cash_sum,bill_BL_cardM__borrow_cash_max,bill_BL_cardM__borrow_cash_median,bill_BL_cardM__borrow_cash_min
userID,,,,,,,,,,,,,,,,,,,,,
2,7,96.598251,21.580308,17.389606,13.799750,0.000000,97.419017,21.600903,17.402921,13.917002,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,6.792245,6.792245,3.396123,0.0
4,20,144.813292,19.483781,0.000000,7.240665,0.000000,105.148045,19.483836,0.000000,5.257402,...,0.0,0.0,4.564541,2.457768,2.106773,0.0,23.357756,18.767298,4.590458,0.0
6,16,314.927082,20.681443,19.638586,19.682943,18.654047,270.925010,21.177840,18.997091,16.932813,...,0.0,0.0,12.581777,12.581777,0.000000,0.0,39.837180,20.559057,19.278123,0.0
7,19,267.402239,19.861357,19.143810,14.073802,0.000000,342.363130,21.135796,20.664428,18.019112,...,0.0,0.0,20.226757,20.226757,0.000000,0.0,38.738568,19.460445,9.639061,0.0
9,12,241.771564,21.330732,20.248808,20.147630,18.587637,243.389228,21.622615,20.299273,20.282436,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,39.654859,20.376736,19.278123,0.0


In [177]:
CF.Calnan(bill_before_stats)["nancount"].value_counts()

0    99
Name: nancount, dtype: int64

In [179]:
gb11 = bill_detail_L_after.groupby(by="userID")
bill_after_recordcount = gb11.size()
bill_after_recordcount.name = "bill_after_recordcount"
bill_after_recordcount = pd.DataFrame(bill_after_recordcount)


bill_detail_L_after_pick = bill_detail_L_after.drop(["bill_time","bankID","loan_time","repay_status"],axis = 1)
gb12 = bill_detail_L_after_pick.groupby(by="userID")
bill_detail_L_after_pick_stats = gb12.aggregate([np.sum,np.max,np.median,np.mean,np.min])
#print("shape:  ",bill_detail_L_after_pick_stats.shape)
# bill_detail_L_after_pick_stats.head()

# 改列名
# AL 代表after loan，放款后。 
# bill_AL_last_bill_amount_sum 表示 放款后某用户ID的上期账单金额列的所有条目的 值的 和。
# bill_detail_L_after_pick_stats 有 55列，分别是 （训练集+未知数据集里）有放款后bill_detail信息的用户在放款后的
# '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
# '循环利息','可用余额','预借现金额度' 的 sum, max, median, mean, min.共11*5 = 55 列。
# 本表无缺失。

L1 = ['sum','max',"median",'mean','min']
L2 = ['last_bill_amount','last_repay_amount','credit_limit','current_bill_balance','current_bill_minrepay','consume_num','current_bill_amount','adjust_amount','cyclic_interest','available_balance','borrow_cash']
L = ['_'.join(['bill_AL',i,j]) for i in L2 for j in L1]    

bill_detail_L_after_pick_stats.columns = L
#print("shape:  ",bill_detail_L_after_pick_stats.shape)
# bill_detail_L_after_pick_stats.head()

# “repay_status”这列，是“还款状态”，因为大部分是0（正常还款），小部分是1（逾期未还）。 
# 对每个用户ID在其所有银行的所有银行卡 求 此还款状态列的 和（表示逾期次数），
# 并求此还款状态列的数量（表示收集到的所有账单次数），两者相除 表示每个用户ID在其所有银行的所有银行卡的所有收集到的账期内 
# 逾期的概率（放款后）。
# bill_AL_overdue_percent 表示 （训练集+未知数据集里）有放款后bill_detail信息的用户在放款后的 “账期逾期概率”。
# 此表无缺失。

tmp58 = bill_detail_L_after[['userID','repay_status']].groupby(by = 'userID').aggregate([np.sum, np.size])
tmp58.columns = ['tmp_sum','tmp_count'] 
tmp58 = tmp58.assign(bill_AL_overdue_percent = tmp58['tmp_sum']/tmp58['tmp_count']) # assign函数为 tmp33增加一列。
bill_AL_overdue_percent = pd.DataFrame(tmp58['bill_AL_overdue_percent'])
#print("shape:  ",bill_AL_overdue_percent.shape)
# bill_AL_overdue_percent.head()

# 可用余额 <0 表示超额消费，用每个用户放款后 可用余额<0 的次数 比上 此用户总的记录数，得到 放款后的超额消费率。
# bill_AL_excess_percent 表示（训练集+未知数据集里）有放款后bill_detail信息的用户在放款后的 超额消费率。
# 本表无缺失。

tmp59 = bill_detail_L_after[['userID','available_balance']]
tmp59["excess"] = tmp59['available_balance']<0
tmp60 = tmp59.groupby("userID").aggregate({"available_balance":np.size, "excess":np.sum})
tmp60.columns = ["num","excess_num"]
tmp60 = tmp60.assign(bill_AL_excess_percent = tmp60["excess_num"]/tmp60["num"])
bill_AL_excess_percent = pd.DataFrame(tmp60["bill_AL_excess_percent"])
#print("shape:  ",bill_AL_excess_percent.shape)
bill_AL_excess_percent.head()

# 本期账单余额 <0 表示溢缴，用每个用户放款后 本期账单余额 <0  的次数 比上 此用户总的记录数，得到 放款后的溢缴率。
# bill_AL_overpay_percent 表示（训练集+未知数据集里）有放款后bill_detail信息的用户在放款后的 溢缴率。
# 本表无缺失。

tmp61 = bill_detail_L_after[['userID','current_bill_balance']]
tmp61["overpay"] = tmp61['current_bill_balance']<0
tmp62 = tmp61.groupby("userID").aggregate({"current_bill_balance":np.size, "overpay":np.sum})
tmp62.columns = ["num","overpay_num"]
tmp62 = tmp62.assign(bill_AL_overpay_percent = tmp62["overpay_num"]/tmp62["num"])
bill_AL_overpay_percent = pd.DataFrame(tmp62["bill_AL_overpay_percent"])



tmp63 = bill_detail_L_after[[ 'userID', 'bankID','last_bill_amount','last_repay_amount','current_bill_balance',
       'current_bill_minrepay', 'consume_num', 'current_bill_amount',
       'adjust_amount', 'cyclic_interest', 'available_balance', 'borrow_cash']]
tmp64 = tmp63.groupby(['userID', 'bankID']).aggregate(np.mean)
tmp64.columns = ['bill_AL_cardM_'+i for i in tmp64.columns]  # 改列名。AL代表放款后，cardM代表 每张卡的平均值
bill_AL_cardM = tmp64
tmp65 = bill_AL_cardM.reset_index(level=-1).drop("bankID",axis=1).groupby("userID")
bill_AL_cardM_stats = tmp65.aggregate([np.sum,np.max,np.median,np.min])

L1 = ['sum','max',"median",'min']  # 改列名
L2 = ['last_bill_amount','last_repay_amount','current_bill_balance',
       'current_bill_minrepay', 'consume_num', 'current_bill_amount',
       'adjust_amount', 'cyclic_interest', 'available_balance', 'borrow_cash']
L = ['_'.join(['bill_AL_cardM_',i,j]) for i in L2 for j in L1]    
bill_AL_cardM_stats.columns = L


tmp66 = bill_after_recordcount.join(bill_detail_L_after_pick_stats)
tmp67 = tmp66.join(bill_AL_overdue_percent)
tmp68 = tmp67.join(bill_AL_excess_percent)
tmp69 = tmp68.join(bill_AL_overpay_percent)
bill_after_stats = tmp69.join(bill_AL_cardM_stats)
print("shape:  ", bill_after_stats.shape)
bill_after_stats.head()


C:\Users\shuangjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\shuangjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


shape:   (42758, 99)


,bill_after_recordcount,bill_AL_last_bill_amount_sum,bill_AL_last_bill_amount_max,bill_AL_last_bill_amount_median,bill_AL_last_bill_amount_mean,bill_AL_last_bill_amount_min,bill_AL_last_repay_amount_sum,bill_AL_last_repay_amount_max,bill_AL_last_repay_amount_median,bill_AL_last_repay_amount_mean,...,bill_AL_cardM__cyclic_interest_median,bill_AL_cardM__cyclic_interest_min,bill_AL_cardM__available_balance_sum,bill_AL_cardM__available_balance_max,bill_AL_cardM__available_balance_median,bill_AL_cardM__available_balance_min,bill_AL_cardM__borrow_cash_sum,bill_AL_cardM__borrow_cash_max,bill_AL_cardM__borrow_cash_median,bill_AL_cardM__borrow_cash_min
userID,,,,,,,,,,,,,,,,,,,,,
2,12,186.611583,21.560264,20.552983,15.550965,0.0,179.694984,21.693954,19.249254,14.974582,...,0.0,0.0,0.000000,0.000000,0.0,0.0,20.376736,20.376736,0.000000,0.000000
4,16,116.837059,19.638332,0.000000,7.302316,0.0,73.005098,18.892461,0.000000,4.562819,...,0.0,0.0,0.000000,0.000000,0.0,0.0,18.767298,18.767298,0.000000,0.000000
7,20,378.804298,21.487808,19.747997,18.940215,0.0,386.857484,21.729404,20.324660,19.342874,...,0.0,0.0,19.558163,19.558163,0.0,0.0,39.158416,19.697971,9.730222,0.000000
8,4,0.000000,0.000000,0.000000,0.000000,0.0,70.682736,17.670684,17.670684,17.670684,...,0.0,0.0,0.000000,0.000000,0.0,0.0,20.376736,20.376736,20.376736,20.376736
10,48,910.681152,20.663117,19.779739,18.972524,0.0,896.187482,20.671592,19.748127,18.670573,...,0.0,0.0,0.000000,0.000000,0.0,0.0,19.278123,19.278123,0.000000,0.000000


In [180]:
CF.Calnan(bill_after_stats)["nancount"].value_counts()

0    99
Name: nancount, dtype: int64

In [182]:
# 统计所有时间（不论放款前、放款后、时间已知未知）



# 统计所有时间，每个用户拥有的 bill_detail 条目记录数，并把它起名为 bill_all_recordcount
# 由 shape 可知，共有 66584 个用户有 所有时间 bill_detail 信息。
# 此表无缺失。

gb11 = bill_detail_L.groupby(by="userID")
bill_all_recordcount = gb11.size()
bill_all_recordcount.name = "bill_all_recordcount"
bill_all_recordcount = pd.DataFrame(bill_all_recordcount)
#print("shape:  ",bill_all_recordcount.shape)
# bill_all_recordcount.head()

# 对于所有时间行条目，删掉 “银行ID” 这列，对每个用户ID在所有银行办理的全部银行卡 进行 groupby 统计。
# 同时还删掉了"bill_time","loan_time","repay_status"这三列。剩下的都是数值型列，值的加和有意义。
# 注意，其实比赛提供的数据集中，一行就是一个账期信息。
# pick 代表挑选特征。
# bill_detail_L_all_pick 包含了 '用户ID', '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额',
# '消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度' 这些列。 
# 注意，我本来想填上 np.std的，但是会产生缺失值。比如有某个用户，只有一行记录，且在一个字段上值为0，则对这个字段求std时就是缺失的。
# 本表无缺失。

bill_detail_L_all_pick = bill_detail_L.drop(["bill_time","bankID","loan_time","repay_status"],axis = 1)
gb12 = bill_detail_L_all_pick.groupby(by="userID")
bill_detail_L_all_pick_stats = gb12.aggregate([np.sum,np.max,np.median,np.mean,np.min])
#print("shape:  ",bill_detail_L_all_pick_stats.shape)
# bill_detail_L_all_pick_stats.head()

# 改列名
# ALL 代表ALL time，所有时间。 
# bill_ALL_last_bill_amount_sum 表示 所有时间某用户ID的上期账单金额列的所有条目的 值的 和。
# bill_detail_L_all_pick_stats 有 55列，分别是 （训练集+未知数据集里）有所有时间bill_detail信息的用户在所有时间的
# '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额',
# '循环利息','可用余额','预借现金额度' 的 sum, max, median, mean, min.共11*5 = 55 列。
# 本表无缺失。

L1 = ['sum','max',"median",'mean','min']
L2 = ['last_bill_amount','last_repay_amount','credit_limit','current_bill_balance','current_bill_minrepay','consume_num','current_bill_amount','adjust_amount','cyclic_interest','available_balance','borrow_cash']
L = ['_'.join(['bill_ALL',i,j]) for i in L2 for j in L1]    

bill_detail_L_all_pick_stats.columns = L
#print("shape:  ",bill_detail_L_all_pick_stats.shape)
# bill_detail_L_all_pick_stats.head()

# “repay_status”这列，是“还款状态”，因为大部分是0（正常还款），小部分是1（逾期未还）。 
# 对每个用户ID在其所有银行的所有银行卡 求 此还款状态列的 和（表示逾期次数），
# 并求此还款状态列的数量（表示收集到的所有账单次数），两者相除 表示每个用户ID在其所有银行的所有银行卡的所有收集到的账期内 
# 逾期的概率（所有时间）。
# bill_ALL_overdue_percent 表示 （训练集+未知数据集里）有所有时间bill_detail信息的用户在所有时间的 “账期逾期概率”。
# 此表无缺失。

tmp58 = bill_detail_L[['userID','repay_status']].groupby(by = 'userID').aggregate([np.sum, np.size])
tmp58.columns = ['tmp_sum','tmp_count'] 
tmp58 = tmp58.assign(bill_ALL_overdue_percent = tmp58['tmp_sum']/tmp58['tmp_count']) # assign函数为 tmp33增加一列。
bill_ALL_overdue_percent = pd.DataFrame(tmp58['bill_ALL_overdue_percent'])
#print("shape:  ",bill_ALL_overdue_percent.shape)
# bill_ALL_overdue_percent.head()

# 可用余额 <0 表示超额消费，用每个用户所有时间 可用余额<0 的次数 比上 此用户总的记录数，得到 所有时间的超额消费率。
# bill_ALL_excess_percent 表示（训练集+未知数据集里）有所有时间bill_detail信息的用户在所有时间的 超额消费率。
# 本表无缺失。

tmp59 = bill_detail_L[['userID','available_balance']]
tmp59["excess"] = tmp59['available_balance']<0
tmp60 = tmp59.groupby("userID").aggregate({"available_balance":np.size, "excess":np.sum})
tmp60.columns = ["num","excess_num"]
tmp60 = tmp60.assign(bill_ALL_excess_percent = tmp60["excess_num"]/tmp60["num"])
bill_ALL_excess_percent = pd.DataFrame(tmp60["bill_ALL_excess_percent"])
#print("shape:  ",bill_ALL_excess_percent.shape)
bill_ALL_excess_percent.head()

# 本期账单余额 <0 表示溢缴，用每个用户放款后 本期账单余额 <0  的次数 比上 此用户总的记录数，得到 所有时间的溢缴率。
# bill_ALL_overpay_percent 表示（训练集+未知数据集里）有所有时间bill_detail信息的用户在所有时间的 溢缴率。
# 本表无缺失。

tmp61 = bill_detail_L[['userID','current_bill_balance']]
tmp61["overpay"] = tmp61['current_bill_balance']<0
tmp62 = tmp61.groupby("userID").aggregate({"current_bill_balance":np.size, "overpay":np.sum})
tmp62.columns = ["num","overpay_num"]
tmp62 = tmp62.assign(bill_ALL_overpay_percent = tmp62["overpay_num"]/tmp62["num"])
bill_ALL_overpay_percent = pd.DataFrame(tmp62["bill_ALL_overpay_percent"])
#print("shape:  ",bill_ALL_overpay_percent.shape)
# bill_ALL_overpay_percent.head()

# 假设每个用户在每个银行有一张信用卡。每个用户的每个银行ID对应多条记录，求他们在 '上期账单金额', '上期还款金额','本期账单余额',
# '本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度'的平均值，
# 作为所有时间，每个用户的每个银行的那张卡 平均一个账期 在这些字段上的均值。每个用户有几张卡，那在某个字段上就有几个均值。
# 然后，用 sum, max, median, min聚合函数（千万不要用 mean，会和bill_ALL_XXX_mean重合）对同一个用户的几个均值进行聚合，
# 既得到 所有时间，每个用户平均每个卡上，平均一个账期内，在这些字段上的 和、最大值、中值、最小值。

# bill_ALL_cardM 是以 userID,bankID 为双索引，展示（训练集+未知数据集里）有所有时间bill_detail信息的用户在所有时间的 每个用户
# 的每个银行卡上的 某某字段的均值。字段：'上期账单金额', '上期还款金额','本期账单余额',
# '本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度'。
# bill_ALL_cardM_stats 是以 userID为索引，展示（训练集+未知数据集里）有所有时间bill_detail信息的用户在所有时间的 每个用户
# 的所有银行卡上的 某字段均值 的 sum, max, median, min。字段：'上期账单金额', '上期还款金额','本期账单余额',
# '本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度'。
# ALL代表所有时间、cardM 代表每个用户的属于同一张卡的某字段的均值，bill_ALL_cardM_stats 是对每个用户的几张卡的这些均值（在
# 一个字段下，此用户有几张卡，就有几个均值）的sum, max, median, min。
# 此表无缺失。


tmp63 = bill_detail_L[[ 'userID', 'bankID','last_bill_amount','last_repay_amount','current_bill_balance',
       'current_bill_minrepay', 'consume_num', 'current_bill_amount',
       'adjust_amount', 'cyclic_interest', 'available_balance', 'borrow_cash']]
tmp64 = tmp63.groupby(['userID', 'bankID']).aggregate(np.mean)
tmp64.columns = ['bill_ALL_cardM_'+i for i in tmp64.columns]  # 改列名。ALL代表放款前，cardM代表 每张卡的平均值
bill_ALL_cardM = tmp64
tmp65 = bill_ALL_cardM.reset_index(level=-1).drop("bankID",axis=1).groupby("userID")
bill_ALL_cardM_stats = tmp65.aggregate([np.sum,np.max,np.median,np.min])

L1 = ['sum','max',"median",'min']  # 改列名
L2 = ['last_bill_amount','last_repay_amount','current_bill_balance',
       'current_bill_minrepay', 'consume_num', 'current_bill_amount',
       'adjust_amount', 'cyclic_interest', 'available_balance', 'borrow_cash']
L = ['_'.join(['bill_ALL_cardM_',i,j]) for i in L2 for j in L1]    
bill_ALL_cardM_stats.columns = L
#print("shape:  ",bill_ALL_cardM_stats.shape)
# bill_ALL_cardM_stats.head()



# 将 5 张表依 userID索引 join  起来，得到 bill_all_stats表。

# 此步得到的 bill_all_stats 的 shape为：(66584, 99)，共 99 个字段，分别为：
# 注意，所谓的每个用户是 训练集和未知数据集里有bill_detail信息的所有用户。

# bill_all_recordcount                          所有时间，每个用户拥有的 bill_detail条目记录数
# bill_ALL_last_XXX_sum/max/median/mean/min     所有时间，每个用户的所有记录条目在某个字段上的sum/max/median/mean/min，这些字段XXX包括：
#                                                      '上期账单金额', '上期还款金额','信用卡额度','本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度' 
# bill_ALL_overdue_percent      所有时间，每个用户的 “账期逾期概率”
# bill_ALL_excess_percent       所有时间，每个用户的 超额消费率
# bill_ALL_overpay_percent      所有时间，每个用户的 溢缴率
# bill_ALL_cardM__XXX_sum/max/median/min         所有时间，每个用户的所有银行卡上的 某字段均值 的 sum, max, median, min。
#                                               字段：'上期账单金额', '上期还款金额','本期账单余额','本期账单最低还款额','消费笔数','本期账单金额','调整金额','循环利息','可用余额','预借现金额度'

# 一共 1 + 55 + 3 + 40 = 99 个特征。
# （本表没有缺失值）。


tmp66 = bill_all_recordcount.join(bill_detail_L_all_pick_stats)
tmp67 = tmp66.join(bill_ALL_overdue_percent)
tmp68 = tmp67.join(bill_ALL_excess_percent)
tmp69 = tmp68.join(bill_ALL_overpay_percent)
bill_all_stats = tmp69.join(bill_ALL_cardM_stats)
print("shape:  ", bill_all_stats.shape)
bill_all_stats.head()

C:\Users\shuangjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\shuangjun\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


shape:   (66584, 99)


,bill_all_recordcount,bill_ALL_last_bill_amount_sum,bill_ALL_last_bill_amount_max,bill_ALL_last_bill_amount_median,bill_ALL_last_bill_amount_mean,bill_ALL_last_bill_amount_min,bill_ALL_last_repay_amount_sum,bill_ALL_last_repay_amount_max,bill_ALL_last_repay_amount_median,bill_ALL_last_repay_amount_mean,...,bill_ALL_cardM__cyclic_interest_median,bill_ALL_cardM__cyclic_interest_min,bill_ALL_cardM__available_balance_sum,bill_ALL_cardM__available_balance_max,bill_ALL_cardM__available_balance_median,bill_ALL_cardM__available_balance_min,bill_ALL_cardM__borrow_cash_sum,bill_ALL_cardM__borrow_cash_max,bill_ALL_cardM__borrow_cash_median,bill_ALL_cardM__borrow_cash_min
userID,,,,,,,,,,,,,,,,,,,,,
2,20,300.599440,21.580308,19.254119,15.029972,0.000000,294.503607,21.693954,18.222086,14.725180,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,16.301389,16.301389,0.000000,0.000000
3,2,36.733542,18.371030,18.366771,18.366771,18.362512,37.160348,19.495329,18.580174,18.580174,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
4,36,261.650351,19.638332,0.000000,7.268065,0.000000,178.153143,19.483836,0.000000,4.948698,...,0.0,0.0,2.608309,1.404439,1.20387,0.0,21.062527,18.767298,2.295229,0.000000
5,2,41.358453,20.681871,20.679226,20.679226,20.676582,37.079520,18.560684,18.539760,18.539760,...,0.0,0.0,0.000000,0.000000,0.00000,0.0,19.971271,19.971271,19.971271,19.971271
6,41,821.750504,21.198821,20.104726,20.042695,18.654047,710.899003,21.177840,18.949619,17.339000,...,0.0,0.0,6.378576,6.378576,0.00000,0.0,39.978219,20.559057,19.419162,0.000000


In [183]:
CF.Calnan(bill_all_stats)["nancount"].value_counts()

0    99
Name: nancount, dtype: int64

In [185]:
tmp34 = bill_detail_L[['userID','bankID']].drop_duplicates()
card_count = tmp34.groupby('userID').size()
card_count.name = 'bill_card_count'
bill_card_count = pd.DataFrame(card_count)
print("shape:  ",bill_card_count.shape)
bill_card_count.head()

shape:   (66584, 1)


,bill_card_count
userID,
2,3
3,1
4,3
5,1
6,3


In [187]:
tmp37 = bill_detail_L[['userID','bankID','credit_limit']].groupby(['userID','bankID']).aggregate(np.max) # 获得了 每个用户在每个银行的 最大信用额度。
tmp38 = tmp37.reset_index(level = -1).drop("bankID",axis=1).groupby("userID").aggregate(np.sum)
tmp38.columns = ["bill_credit_limit_cardsum"]
bill_credit_limit_cardsum = tmp38
print("shape:  ",bill_credit_limit_cardsum.shape)
bill_credit_limit_cardsum.head()

shape:   (66584, 1)


,bill_credit_limit_cardsum
userID,
2,61.523250
3,18.361833
4,59.110850
5,20.664418
6,62.615441


In [188]:
CF.Calnan(bill_credit_limit_cardsum)

,column_name,nancount,nanpercent
0,bill_credit_limit_cardsum,0,0.0


In [189]:
tmp70 = bill_all_stats.join(bill_before_stats)
tmp71 = tmp70.join(bill_after_stats)
tmp72 = tmp71.join(bill_card_count)
bill_detail_stats = tmp72.join(bill_credit_limit_cardsum).reset_index()

print("shape:  ", bill_detail_stats.shape)
bill_detail_stats.head()

shape:   (66584, 300)


,userID,bill_all_recordcount,bill_ALL_last_bill_amount_sum,bill_ALL_last_bill_amount_max,bill_ALL_last_bill_amount_median,bill_ALL_last_bill_amount_mean,bill_ALL_last_bill_amount_min,bill_ALL_last_repay_amount_sum,bill_ALL_last_repay_amount_max,bill_ALL_last_repay_amount_median,...,bill_AL_cardM__available_balance_sum,bill_AL_cardM__available_balance_max,bill_AL_cardM__available_balance_median,bill_AL_cardM__available_balance_min,bill_AL_cardM__borrow_cash_sum,bill_AL_cardM__borrow_cash_max,bill_AL_cardM__borrow_cash_median,bill_AL_cardM__borrow_cash_min,bill_card_count,bill_credit_limit_cardsum
0,2,20,300.599440,21.580308,19.254119,15.029972,0.000000,294.503607,21.693954,18.222086,...,0.0,0.0,0.0,0.0,20.376736,20.376736,0.0,0.0,3,61.523250
1,3,2,36.733542,18.371030,18.366771,18.366771,18.362512,37.160348,19.495329,18.580174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,18.361833
2,4,36,261.650351,19.638332,0.000000,7.268065,0.000000,178.153143,19.483836,0.000000,...,0.0,0.0,0.0,0.0,18.767298,18.767298,0.0,0.0,3,59.110850
3,5,2,41.358453,20.681871,20.679226,20.679226,20.676582,37.079520,18.560684,18.539760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,20.664418
4,6,41,821.750504,21.198821,20.104726,20.042695,18.654047,710.899003,21.177840,18.949619,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,62.615441


# 5. 汇总表与缺失值填充

1. user_info(69495*6)、loan_time(69495*2)、overdue(69495*2)三者均包含训练集+未知样本集，前两者无缺失，后者在未知样本的target上缺失。  bank_detail_stats(10003*8)、browse_history_stats(59327*8)、bill_detail_stats(66584*300)三者均包含训练集+未知样本集，前者无缺失、后两者都有缺失。训练集+未知样本集共 69495人，其中训练集 55596人，未知样本集 13899人。
2. 将以上六张表依userID链接到一起，起名 data_origin_1（包含训练集+未知样本集） 。
3. trainuser_without_bankbrowsebill、bank_train_outincome_ratio_outlier、bank_train_income_sum_outlier、bank_train_outcome_sum_outlier、browse_train_allrecordcount_outlier 是各个阶段找出的“训练集”里的异常点，需要删除。它们都是 userID，有的是set格式，有的是index格式，需要汇总去重（唯一化）后，从训练集中删去。注意，测试集里肯定也有异常点，但测试集不能碰。只能删掉训练集里的异常userID。
4. user_info、loan_time、overdue 三部分，user_info都是类别变量，应该onehot。loan_time只有一个连续数值变量，暂时保持不变。（后期可能要标准化），overdue不能处理。
5. bank_detail_stats部分（无缺失）：因为只有14.39%的用户有 bank 信息，太少，所以这样处理：对于 条目数、收入sum、收入count、支出sum、支出count、支出占收入比 这几个字段，都是数值型列，先用MDLP最优分箱离散化，然后把总数据集中没有 bank 信息的用户记为“缺失箱”即可。但需要注意，应该先分离出训练集，然后对训练集的这些字段进行最优分箱，然后把分箱节点应用到未知数据集上对其分箱。然后onehot（这时可以训练集和未知数据集一起）。 而 是否有工资信息 字段，是类别变量，已有工资信息记为2、没有工资记为1、总数据集中没有 bank 信息的用户记为0“缺失”即可。然后onehot（训练集和未知数据集可以一起）。
6. browse_history_stats部分（有缺失）：因为有85.37%的用户有 browse 信息，较多，所以这样处理：此部分的所有列都是数值型列（除userID外），所以缺失值用算法填充（算法填充时，用训练集得到的模型，填充训练集的缺失和未知样本集的缺失）。
7. bill_detail_stats部分（有缺失）：因为有96.15%的用户有bill信息，比较多，所以这样处理：此部分的所有列都是数值型列（除userID外），所以缺失值用算法填充（算法填充时，用训练集得到的模型，填充训练集的缺失和未知样本集的缺失）。


## 5.1 预处理

In [190]:
# 将 user_info、bank_detail_stats、browse_history_stats、bill_detail_stats、loan_time、overdue五张表 merge，
# 得到 data_origin_1.


tmp73 = pd.merge(user_info,bank_detail_stats,how="left",on="userID")
tmp74 = pd.merge(tmp73,browse_history_stats,how="left",on="userID")
tmp75 = pd.merge(tmp74,bill_detail_stats,how="left",on="userID")
tmp76 = pd.merge(tmp75,loan_time,how="left",on="userID")
data_origin_1 = pd.merge(tmp76,overdue,how="left",on="userID")
print("shape of data_origin_1: ", data_origin_1.shape)
data_origin_1.head(5)

shape of data_origin_1:  (69495, 390)


,userID,gender,career,education,marriage,residence,bank_detail_record_count,bank_income_sum,bank_income_count,bank_outcome_sum,...,bill_AL_cardM__available_balance_median,bill_AL_cardM__available_balance_min,bill_AL_cardM__borrow_cash_sum,bill_AL_cardM__borrow_cash_max,bill_AL_cardM__borrow_cash_median,bill_AL_cardM__borrow_cash_min,bill_card_count,bill_credit_limit_cardsum,loan_time,target
0,1,1,2,3,1,3,86.0,480.692762,35.0,676.025269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5914855887,0.0
1,2,1,2,3,2,1,NaN,NaN,NaN,NaN,...,0.0,0.0,20.376736,20.376736,0.0,0.0,3.0,61.523250,5914855887,0.0
2,3,1,4,4,1,4,679.0,2278.873446,172.0,4985.957607,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,18.361833,5914855887,0.0
3,4,1,4,4,3,2,291.0,1164.342384,96.0,2129.425722,...,0.0,0.0,18.767298,18.767298,0.0,0.0,3.0,59.110850,5914855887,1.0
4,5,1,2,2,3,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,20.664418,5914855887,0.0


In [191]:
# 计算 trainuser_without_bankbrowsebill、bank_train_outincome_ratio_outlier、bank_train_income_sum_outlier、
# bank_train_outcome_sum_outlier、browse_train_allrecordcount_outlier 内含的训练集异常点的userID并集，共215个异常点。
# 从 data_origin_1 中删去这些 异常点，得到 data_origin_2.

tmp77 = set(trainuser_without_bankbrowsebill)
tmp78 = set(bank_train_outincome_ratio_outlier)
tmp79 = set(bank_train_income_sum_outlier)
tmp80 = set(bank_train_outcome_sum_outlier)
tmp81 = set(browse_train_allrecordcount_outlier)
outlier_userID = tmp77|tmp78|tmp79|tmp80|tmp81
print("The number of outlier is: ", len(outlier_userID))

data_origin_2 = data_origin_1.set_index("userID").drop(list(outlier_userID))
print("shape of data_origin_2: ", data_origin_2.shape)
data_origin_2.head()

The number of outlier is:  215
shape of data_origin_2:  (69280, 389)


,gender,career,education,marriage,residence,bank_detail_record_count,bank_income_sum,bank_income_count,bank_outcome_sum,bank_outcome_count,...,bill_AL_cardM__available_balance_median,bill_AL_cardM__available_balance_min,bill_AL_cardM__borrow_cash_sum,bill_AL_cardM__borrow_cash_max,bill_AL_cardM__borrow_cash_median,bill_AL_cardM__borrow_cash_min,bill_card_count,bill_credit_limit_cardsum,loan_time,target
userID,,,,,,,,,,,,,,,,,,,,,
1,1,2,3,1,3,86.0,480.692762,35.0,676.025269,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5914855887,0.0
2,1,2,3,2,1,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,20.376736,20.376736,0.0,0.0,3.0,61.523250,5914855887,0.0
3,1,4,4,1,4,679.0,2278.873446,172.0,4985.957607,507.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,18.361833,5914855887,0.0
4,1,4,4,3,2,291.0,1164.342384,96.0,2129.425722,195.0,...,0.0,0.0,18.767298,18.767298,0.0,0.0,3.0,59.110850,5914855887,1.0
5,1,2,2,3,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,20.664418,5914855887,0.0


In [192]:
# 得到去掉异常点后的训练集的 userID构成的列表 userIDtrain，现在训练集共 55381 个用户。
# 得到未知样本集的 userID构成的列表 userIDtest，现在未知样本集共 13899 个用户。
# 从 data_origin_2 中提取出 训练集 data_origin_2_train，未知样本集 data_origin_2_test。


userIDtrain = list(set(overdue_train.reset_index()['userID'])-outlier_userID)  # 得到了删去异常点后的训练集的 用户ID构成的列表。用来提取 训练集。
userIDtest = list(overdue_test.reset_index()['userID'])  # 得到了未知样本集的 用户ID构成的集合。用来提取 未知样本集。
print("The number of train without outliers is: ",len(userIDtrain))
print("The number of test is: ",len(userIDtest))

data_origin_2_train = data_origin_2.loc[userIDtrain,:]
print("shape of data_origin_2_train is:",data_origin_2_train.shape )
data_origin_2_test = data_origin_2.loc[userIDtest,:]
print("shape of data_origin_2_test is:",data_origin_2_test.shape )

The number of train without outliers is:  55381
The number of test is:  13899
shape of data_origin_2_train is: (55381, 389)
shape of data_origin_2_test is: (13899, 389)


## 5.2 bank_detail 分箱处理

&ensp;&ensp;bank_detail_stats部分：对于 条目数、收入sum、收入count、支出sum、支出count、支出占收入比 这几个字段，都是数值型列，先用MDLP最优分箱离散化，然后把总数据集中没有 bank 信息的用户记为“缺失箱”即可。但需要注意，应该先分离出训练集，然后对训练集的这些字段进行最优分箱，然后把分箱节点应用到未知数据集上对其分箱。然后onehot（这时可以训练集和未知数据集一起）。 而 是否有工资信息 字段，是类别变量，已有工资信息记为2、没有工资记为1、总数据集中没有 bank 信息的用户记为0“缺失”即可。然后onehot（训练集和未知数据集可以一起）。

In [193]:
# bank_detail_stats 的 shape为：(10003, 8)，共 8个字段，分别为：
# userID                     训练集和未知数据集里有bank_detail信息的所有用户
# bank_detail_record_count   每个userID拥有的bank_detail信息的条目数。
# bank_income_sum            每个userID的 总收入。
# bank_income_count          每个userID的 收入笔数。
# bank_outcome_sum           每个userID的 总支出。
# bank_outcome_count         每个userID的 支出笔数。
# bank_salary_record         每个userID 是否有 工资信息（有工资信息的记为2，没工资信息的记为1）
# bank_out/income            每个userID的 支出占收入比


In [194]:
# c 是从 bank_detail里提取出来的要进行分箱的列名，共6个：条目数、收入sum、收入count、支出sum、支出count、支出占收入比。
# 调用自定义函数 CF.bank_detail_change，实现对这 6 个列的最优分箱和转化。（对训练集最优分箱，并对训练集进行转换。然后把训练集
# 的最优分箱的各箱值域应用到未知样本集上，对未知样本集进行转换。转换的结果是，对一个变量，第一个箱的样本的值被转化为0，第二个
# 箱的为1，第三个箱的为2....）。此函数接收了训练集data_origin_2_train和未知样本集data_origin_2_test，但只对它们的bank_detail
# 的那六列进行转换，返回转换后的bank_detail部分的训练集 和 未知样本集。
# 正则参数a=0.0015是反复试验得到的，能够把这六列的每列分成两到四箱。
# 函数运行过程中会打印出对每个列字段的最优分箱建议，包括MDLP算法的 Ps 和 binsinfo。以及 binsvalue（各箱值域）。

t0 = time.time()
c = ["bank_detail_record_count" ,"bank_income_sum", "bank_income_count" ,"bank_outcome_sum" ,"bank_outcome_count","bank_out/income" ]
bank_detail_train_changed1,bank_detail_test_changed1=CF.bank_detail_change(data_origin_2_train.reset_index(), data_origin_2_test.reset_index(), "userID", c, "target", 0.0015)
t1 = time.time()
print(t1-t0)

------- 结果为  bank_detail_record_count  -------
划分点： Ps =  [5203]
各分箱： binsinfo =  [[0, 5203, 0.6600380123872718, 0], [5204, 9136, 0.5818232376090253, 0]]
binsvalue =  [[1.0, 576.0], [576.0, 3810.0]]
------- 结果为  bank_income_sum  -------
划分点： Ps =  [2, 1000, 3974]
各分箱： binsinfo =  [[0, 2, 0.0, 0], [3, 1000, 0.7227289742146383, 0], [1001, 3974, 0.658795866551041, 0], [3975, 9136, 0.5864263486530137, 0]]
binsvalue =  [[0.0, 6.562638000000001], [6.8499669999999995, 473.98908399999976], [474.1277089999999, 1495.6673489999996], [1495.8838430000005, 10994.517722999984]]
------- 结果为  bank_income_count  -------
划分点： Ps =  [3615]
各分箱： binsinfo =  [[0, 3615, 0.6716074371572747, 0], [3616, 9136, 0.5969322933850869, 0]]
binsvalue =  [[0.0, 109.0], [109.0, 900.0]]
------- 结果为  bank_outcome_sum  -------
划分点： Ps =  [3685, 3814]
各分箱： binsinfo =  [[0, 3685, 0.6714073677899599, 0], [3686, 3814, 0.843349289138202, 0], [3815, 9136, 0.5881229453823549, 0]]
binsvalue =  [[0.0, 3189.439852000003], [3189.58034

In [195]:
print("shape of bank_detail_train_changed is : ",bank_detail_train_changed1.shape)
bank_detail_train_changed1.head()

shape of bank_detail_train_changed is :  (55381, 6)


,bank_detail_record_count,bank_income_sum,bank_income_count,bank_outcome_sum,bank_outcome_count,bank_out/income
userID,,,,,,
1,0,2,0,0,0,0
2,2,4,2,3,2,2
3,1,3,1,2,1,0
4,0,2,0,0,0,0
5,2,4,2,3,2,2


In [196]:
print("shape of bank_detail_test_changed is : ",bank_detail_test_changed1.shape)
bank_detail_test_changed1.head()

shape of bank_detail_test_changed is :  (13899, 6)


,bank_detail_record_count,bank_income_sum,bank_income_count,bank_outcome_sum,bank_outcome_count,bank_out/income
userID,,,,,,
55597,0,1,0,0,0,0
55598,2,4,2,3,2,2
55599,1,3,1,2,1,0
55600,2,4,2,3,2,2
55601,2,4,2,3,2,2


In [197]:
# 将 bank_detail_train_changed1 和 bank_detail_test_changed1 上下 concat 得到大表 bank_detail_changed1。（含训练集与未知数据集）
# 我们看到此表里的值不再是连续值，而都是0,1,2,3,4等离散值。

bank_detail_changed1 = pd.concat([bank_detail_train_changed1,bank_detail_test_changed1])
print("shape of bank_detail_changed1 is: ",bank_detail_changed1.shape)
bank_detail_changed1.head()

shape of bank_detail_changed1 is:  (69280, 6)


,bank_detail_record_count,bank_income_sum,bank_income_count,bank_outcome_sum,bank_outcome_count,bank_out/income
userID,,,,,,
1,0,2,0,0,0,0
2,2,4,2,3,2,2
3,1,3,1,2,1,0
4,0,2,0,0,0,0
5,2,4,2,3,2,2


In [198]:
CF.Calnan(bank_detail_changed1)

,column_name,nancount,nanpercent
0,bank_detail_record_count,0,0.0
1,bank_income_sum,0,0.0
2,bank_income_count,0,0.0
3,bank_outcome_sum,0,0.0
4,bank_outcome_count,0,0.0
5,bank_out/income,0,0.0


In [199]:
# 对 bank_detail 里的 是否有工资信息 字段 进行处理。
# 此字段是类别变量，已有工资信息已记为2、没有工资信息的已记为1。而总数据集中没有 bank 信息的用户现在是nan，
# 我需要将之转换为0，代表“缺失”类别。然后onehot。
# 因为此过程只以每个userID为单位进行改变，所以可以训练集和未知数据集一起进行。

tmp82 = pd.DataFrame(data_origin_2["bank_salary_record"])
tmp82.fillna(0,inplace=True)
tmp82 = tmp82.astype(np.int64)

bank_detail_changed = pd.concat([bank_detail_changed1,tmp82],axis=1)  # 将之前分箱后转化的6个列 和 此列 concat，得到7个列
print("shape of bank_detail_changed is: ", bank_detail_changed.shape)
bank_detail_changed.head()

shape of bank_detail_changed is:  (69280, 7)


,bank_detail_record_count,bank_income_sum,bank_income_count,bank_outcome_sum,bank_outcome_count,bank_out/income,bank_salary_record
userID,,,,,,,
1,0,2,0,0,0,0,1
2,2,4,2,3,2,2,0
3,1,3,1,2,1,0,1
4,0,2,0,0,0,0,1
5,2,4,2,3,2,2,0


In [200]:
CF.Calnan(bank_detail_changed)

,column_name,nancount,nanpercent
0,bank_detail_record_count,0,0.0
1,bank_income_sum,0,0.0
2,bank_income_count,0,0.0
3,bank_outcome_sum,0,0.0
4,bank_outcome_count,0,0.0
5,bank_out/income,0,0.0
6,bank_salary_record,0,0.0


In [201]:
# 将 bank_detail_changed 进行 onehot转换。得到的 bank_detail_OHE 是onehot之后的 bank_detail_changed，是ndarray格式。

enc_bank = OneHotEncoder(sparse=False)
bank_detail_OHE = enc_bank.fit_transform(bank_detail_changed)
bank_detail_OHE = bank_detail_OHE.astype(np.int64)   # 转换类型
print("shape of bank_detail_OHE is: ", bank_detail_OHE.shape)
bank_detail_OHE

shape of bank_detail_OHE is:  (69280, 24)


array([[1, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 1, 0, 0],
       [0, 0, 1, ..., 1, 0, 0]], dtype=int64)

In [202]:
# 将 bank_detail_OHE转化为 dataframe 并改列名。

# bank_detail_OHE 是 对bank_detail部分的最终转化版本。其shape是 (69280, 24).
# userID 为 用户ID。
# 其列为:
# bank_detail_record_count_OHE_0/1/2   每个userID拥有的bank_detail信息的条目数。共3列。
# bank_income_sum_OHE_0/1/2/3/4        每个userID的 总收入。共5列。
# bank_income_count_OHE_0/1/2          每个userID的 收入笔数。共3列。
# bank_outcome_sum_OHE_0/1/2/3         每个userID的 总支出。共4列。
# bank_outcome_count_OHE_0/1/2         每个userID的 支出笔数。共3列。
# bank_salary_record_OHE_0/1/2         每个userID 是否有 工资信息（onehot之前，有工资信息的记为2，没工资信息的记为1,没有bank信息的为0）。共3列。
# bank_out/income_OHE_0/1/2            每个userID的 支出占收入比。共3列。
# (注意，本表无缺失。本表包含了训练集和未知样本集。本表是 onehot 后的表)

bank_index = bank_detail_changed.index
bank_columns = bank_detail_changed.columns
L = ['_'.join([i,"OHE",str(k)]) for i,j in zip(bank_columns,enc_bank.n_values_ ) for k in range(j) ]

bank_detail_OHE = pd.DataFrame(bank_detail_OHE, columns=L, index=bank_index)
print("shape of bank_detail_OHE is: ",bank_detail_OHE.shape)
bank_detail_OHE.head()

# L 为创建的列名，相当于：
# for i,j in zip(columns,enc_bank.n_values_):  # columns是未onehot时的原列名。enc_bank.n_values_是每个列onehot后得到的列数。
#     for k in range(j):
#         L.append('_'.join([i,"OHE",str(k)]))

shape of bank_detail_OHE is:  (69280, 24)


,bank_detail_record_count_OHE_0,bank_detail_record_count_OHE_1,bank_detail_record_count_OHE_2,bank_income_sum_OHE_0,bank_income_sum_OHE_1,bank_income_sum_OHE_2,bank_income_sum_OHE_3,bank_income_sum_OHE_4,bank_income_count_OHE_0,bank_income_count_OHE_1,...,bank_outcome_sum_OHE_3,bank_outcome_count_OHE_0,bank_outcome_count_OHE_1,bank_outcome_count_OHE_2,bank_out/income_OHE_0,bank_out/income_OHE_1,bank_out/income_OHE_2,bank_salary_record_OHE_0,bank_salary_record_OHE_1,bank_salary_record_OHE_2
userID,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,1,0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
2,0,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,0,1,1,0,0
3,0,1,0,0,0,0,1,0,0,1,...,0,0,1,0,1,0,0,0,1,0
4,1,0,0,0,0,1,0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
5,0,0,1,0,0,0,0,1,0,0,...,1,0,0,1,0,0,1,1,0,0


In [203]:
CF.Calnan(bank_detail_OHE)

,column_name,nancount,nanpercent
0,bank_detail_record_count_OHE_0,0,0.0
1,bank_detail_record_count_OHE_1,0,0.0
2,bank_detail_record_count_OHE_2,0,0.0
3,bank_income_sum_OHE_0,0,0.0
4,bank_income_sum_OHE_1,0,0.0
5,bank_income_sum_OHE_2,0,0.0
6,bank_income_sum_OHE_3,0,0.0
7,bank_income_sum_OHE_4,0,0.0
8,bank_income_count_OHE_0,0,0.0
9,bank_income_count_OHE_1,0,0.0


## 5.3 user_info

&ensp;&ensp;data_origin_2中，user_info部分无缺失，且对应的都是类别变量，应该onehot。此处做这个工作。另外，因为onehot是针对每个样本的，所以可以训练集和未知数据集一起进行。

In [204]:
data_origin_2.head()

,gender,career,education,marriage,residence,bank_detail_record_count,bank_income_sum,bank_income_count,bank_outcome_sum,bank_outcome_count,...,bill_AL_cardM__available_balance_median,bill_AL_cardM__available_balance_min,bill_AL_cardM__borrow_cash_sum,bill_AL_cardM__borrow_cash_max,bill_AL_cardM__borrow_cash_median,bill_AL_cardM__borrow_cash_min,bill_card_count,bill_credit_limit_cardsum,loan_time,target
userID,,,,,,,,,,,,,,,,,,,,,
1,1,2,3,1,3,86.0,480.692762,35.0,676.025269,51.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5914855887,0.0
2,1,2,3,2,1,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,20.376736,20.376736,0.0,0.0,3.0,61.523250,5914855887,0.0
3,1,4,4,1,4,679.0,2278.873446,172.0,4985.957607,507.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,18.361833,5914855887,0.0
4,1,4,4,3,2,291.0,1164.342384,96.0,2129.425722,195.0,...,0.0,0.0,18.767298,18.767298,0.0,0.0,3.0,59.110850,5914855887,1.0
5,1,2,2,3,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,20.664418,5914855887,0.0


In [205]:
# 把 user_info 进行 onehot转换，得到 user_info_OHE。

# user_info_OHE 是 user_info 部分的完全转化版本，其 shape 是：(69280, 24)
# user_info_OHE的列：

# gender_OHE_0/1/2         每个userID的性别。共3列。
# career_OHE_0/1/2/3/4     每个userID的职业。共5列。
# education_0/1/2/3/4      每个userID的教育。共5列。
# marriage_0/1/2/3/4/5     每个userID的婚姻。共6列。
# residence_0/1/2/3/4      每个userID的户口。共5列。
# （本表无缺失值，已 onehot，包含训练集和未知样本集）

c = ["gender","career","education","marriage","residence"]
tmp83 = data_origin_2[c]
enc_userinfo = OneHotEncoder(sparse=False)
user_info_OHE = enc_userinfo.fit_transform(tmp83)
user_info_OHE = user_info_OHE.astype(np.int64)   # 转换类型


# 改列名
userinfo_index = tmp83.index
userinfo_columns = tmp83.columns
L = ['_'.join([i,"OHE",str(k)]) for i,j in zip(userinfo_columns,enc_userinfo.n_values_ ) for k in range(j)]

user_info_OHE = pd.DataFrame(user_info_OHE, columns=L, index=userinfo_index)
print("shape of user_info_OHE is: ",user_info_OHE.shape)
user_info_OHE.head()

shape of user_info_OHE is:  (69280, 24)


,gender_OHE_0,gender_OHE_1,gender_OHE_2,career_OHE_0,career_OHE_1,career_OHE_2,career_OHE_3,career_OHE_4,education_OHE_0,education_OHE_1,...,marriage_OHE_1,marriage_OHE_2,marriage_OHE_3,marriage_OHE_4,marriage_OHE_5,residence_OHE_0,residence_OHE_1,residence_OHE_2,residence_OHE_3,residence_OHE_4
userID,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
5,0,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [206]:
CF.Calnan(user_info_OHE)

,column_name,nancount,nanpercent
0,gender_OHE_0,0,0.0
1,gender_OHE_1,0,0.0
2,gender_OHE_2,0,0.0
3,career_OHE_0,0,0.0
4,career_OHE_1,0,0.0
5,career_OHE_2,0,0.0
6,career_OHE_3,0,0.0
7,career_OHE_4,0,0.0
8,education_OHE_0,0,0.0
9,education_OHE_1,0,0.0


## 5.4 browse_history_stats 和 bill_detail_stats

&ensp;&ensp;browse_history_stats部分（有缺失）：因为有85.37%的用户有 browse 信息，较多，所以这样处理：此部分的所有列都是数值型列（除userID外），所以缺失值用算法填充（算法填充时，用训练集得到的模型，填充训练集的缺失和未知样本集的缺失）。  

&ensp;&ensp;bill_detail_stats部分（有缺失）：因为有96.15%的用户有bill信息，比较多，所以这样处理：此部分的所有列都是数值型列（除userID外），所以缺失值用算法填充（算法填充时，，用训练集得到的模型，填充训练集的缺失和未知样本集的缺失）。  

&ensp;&ensp;因为自定义函数RFR_fillna在填充时，需要使用完整的列做参考，所以能够依靠的只有 user_info_OHE，bank_detail_OHE，loan_time，因为现在只有他们三个是完整的。我觉得browse_history_stats 和 bill_detail_stats可以分开填充，先填充browse_history_stats，得到完整表后，就可以把browse_history_stats结合到user_info_OHE，bank_detail_OHE，loan_time上，然后一起去进行bill_detail_stats的填充。这样，bill_detail_stats在填充时，可以多依赖一部分。否则，如果browse_history_stats 和 bill_detail_stats一起填充，则它们都只能依赖user_info_OHE，bank_detail_OHE，loan_time。

In [207]:
# 从原表中得到user_info_OHE,bank_detail_OHE,loan_time这三部分，都是完全已知无缺失的，将它们合并得到df_ref1.
# 从原表中得到原来 browse_history_stats对应的 浏览信息部分，它的各个列都是数值型列且是有缺失的，待填充，为 df_withnan1.

tmp84 = data_origin_2.loc[:,"loan_time"]
df_ref1= pd.concat([user_info_OHE,bank_detail_OHE,tmp84],axis=1) 
df_withnan1 = data_origin_2.loc[:,"browse_all_recordcount":"browse_BLbn_10_count"]
print("shape of df_ref1 is:",df_ref1.shape)
print("shape of df_withnan1 is:",df_withnan1.shape)

shape of df_ref1 is: (69280, 49)
shape of df_withnan1 is: (69280, 76)


In [208]:
# 本函数用来依据没有缺失值的表填充有缺失值的表。
# 参数：
# df_ref 是完整的无任何缺失值的列构成的dataframe，以userID为行索引，可以作为算法填充缺失值的参考列。
# df_withnan 是所有有缺失值且需要用算法填充的列构成的dataframe，以userID为行索引，df_withnan必须与df_ref的行数相同！！
# userIDtrain和userIDtest分别是训练集的userID构成的list和未知样本集的userID构成的list。
# 返回值：
# 返回一个dataframe，是 df_ref和已填充的 df_withnan 左右concat之后得到的表，此表是完整的，没有缺失值。

# 基本思路见《SKlearn 缺失值处理方式》




def RFR_fillna(df_ref, df_withnan, userIDtrain, userIDtest):    # 用 随机森林回归算法填充缺失值。
    
    df_fillna = []  # 把 df_withnan中每个填充好的列收集到 df_fillna 里
    
    for i in df_withnan.columns :  # 对 df_withnan 里的每一列进行循环填充
        df = pd.concat([pd.DataFrame(df_withnan.loc[:,i]),df_ref],axis=1)  # 把待填充列 i 与 df_ref 左右连接得到 df，且列 i在最左侧
        
        df_train = df.loc[userIDtrain,:]
        df_test = df.loc[userIDtest,:]
        
        df_train_complete = df_train.loc[df_train[i].notnull(),:]
        df_train_incomplete = df_train.loc[df_train[i].isnull(),:]
        df_test_complete = df_test.loc[df_test[i].notnull(),:]
        df_test_incomplete = df_test.loc[df_test[i].isnull(),:]
        
               
        X_tr = df_train_complete.iloc[:,1:]  
        Y_tr = df_train_complete.iloc[:,0]
        X_te = pd.concat([df_train_incomplete.iloc[:,1:],df_test_incomplete.iloc[:,1:]])
        
        regr = RandomForestRegressor(n_estimators = 20,max_depth=10,random_state=0,n_jobs=-1) # 注意，一定设置random_state，以保证下一次的填充是一样的。
        regr.fit(X_tr,Y_tr)
        index = X_te.index
        tmp1=pd.DataFrame(regr.predict(X_te),index=index,columns=[i]) 
        
        # 将 df_complete的第 i 列和 tmp1 上下连接，得到 column_fillna，它是完整的填充好的第 i 列。
        column_fillna = pd.concat([pd.DataFrame(Y_tr),tmp1,pd.DataFrame(df_test_complete.loc[:,i])]).sort_index() 
        df_fillna.append(column_fillna)  # 将填充好的第 i 列添加到 df_fillna 里收集起来
        
    tmp = pd.concat(df_fillna,axis=1) # tmp是已经填充好缺失值的 df_withnan.
    return pd.concat([df_ref,tmp],axis = 1)  # 将 df_ref 和 tmp 左右结合，返回，即完整的没有缺失值的 整个表。
         

In [209]:
# 使用自定义函数 RFR_fillna，对 browse部分的缺失值进行填充（训练集+未知样本集），得到完全填充的 user_info,bank_detail,loan_time,
# browse_history左右concat构成的表 browse_complete.此表的shape为(69280, 125)，完全无缺失。

t0 = time.time()
browse_complete = RFR_fillna(df_ref1, df_withnan1, userIDtrain, userIDtest)
t1 = time.time()
print("Time: {:.3f} seconds".format(t1-t0))

# 随机森林里 n_estimators =10, max_depth=5时用时 65.7s。 max_depth=10时用时 82s
# 随机森林里 n_estimators =20, max_depth=10时用时 125s。 

Time: 134.067 seconds


In [210]:
print("shape of browse_complete is:",browse_complete.shape)
browse_complete.head()

shape of browse_complete is: (69280, 125)


,gender_OHE_0,gender_OHE_1,gender_OHE_2,career_OHE_0,career_OHE_1,career_OHE_2,career_OHE_3,career_OHE_4,education_OHE_0,education_OHE_1,...,browse_BLbn_5_min,browse_BLbn_6_min,browse_BLbn_7_min,browse_BLbn_10_min,browse_BLbn_1_count,browse_BLbn_4_count,browse_BLbn_5_count,browse_BLbn_6_count,browse_BLbn_7_count,browse_BLbn_10_count
userID,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,1,0,0,0,0,...,42.645163,44.020016,38.170320,148.913129,145.679365,47.431893,36.603805,32.578424,26.439743,21.435619
2,0,1,0,0,0,1,0,0,0,0,...,44.000000,7.000000,38.000000,190.000000,540.000000,150.000000,105.000000,135.000000,105.000000,75.000000
3,0,1,0,0,0,0,0,1,0,0,...,35.000000,50.000000,38.000000,190.000000,174.000000,48.000000,30.000000,24.000000,42.000000,24.000000
4,0,1,0,0,0,0,0,1,0,0,...,35.000000,50.000000,38.000000,190.000000,164.000000,64.000000,36.000000,32.000000,36.000000,24.000000
5,0,1,0,0,0,1,0,0,0,0,...,41.221668,46.914263,38.169541,153.315456,222.445126,97.153795,57.093226,41.796172,43.478430,38.161428


In [211]:
CF.Calnan(browse_complete)["nancount"].value_counts()  # 可见，全部填充完毕，没有空值。

0    125
Name: nancount, dtype: int64

In [212]:
# 现在，用完全已知无缺失的browse_complete作为 df_ref2（包含user_info_OHE,bank_detail_OHE,loan_time,browse_history这四部分）
# 从原表中得到原来 bill_detail_stats对应的 信用卡信息部分，它的各个列都是数值型列且是有缺失的，待填充，为 df_withnan2.

df_ref2= browse_complete
df_withnan2 = data_origin_2.loc[:,"bill_all_recordcount":"bill_credit_limit_cardsum"]
print("shape of df_ref2 is:",df_ref2.shape)
print("shape of df_withnan2 is:",df_withnan2.shape)

shape of df_ref2 is: (69280, 125)
shape of df_withnan2 is: (69280, 299)


In [213]:
# 使用自定义函数 RFR_fillna，对 bill部分的缺失值进行填充（训练集+未知样本集），得到完全填充的 user_info,bank_detail,loan_time,
# browse_history,bill_detail左右concat构成的表 data_complete.此表的shape为，完全无缺失。

t0 = time.time()
data_complete = RFR_fillna(df_ref2, df_withnan2, userIDtrain, userIDtest) 
t1 = time.time()
print(t1-t0)

# 随机森林里 n_estimators =20, max_depth=10时用时 58 min。

4101.832143783569


In [214]:
print("shape of data_complete is:",data_complete.shape)
data_complete.head()

shape of data_complete is: (69280, 424)


,gender_OHE_0,gender_OHE_1,gender_OHE_2,career_OHE_0,career_OHE_1,career_OHE_2,career_OHE_3,career_OHE_4,education_OHE_0,education_OHE_1,...,bill_AL_cardM__available_balance_sum,bill_AL_cardM__available_balance_max,bill_AL_cardM__available_balance_median,bill_AL_cardM__available_balance_min,bill_AL_cardM__borrow_cash_sum,bill_AL_cardM__borrow_cash_max,bill_AL_cardM__borrow_cash_median,bill_AL_cardM__borrow_cash_min,bill_card_count,bill_credit_limit_cardsum
userID,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,1,0,0,0,0,...,1.025837,1.025837,0.644378,0.049019,25.766646,14.131988,11.047256,5.839872,2.897174,44.916014
2,0,1,0,0,0,1,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,20.376736,20.376736,0.000000,0.000000,3.000000,61.523250
3,0,1,0,0,0,0,0,1,0,0,...,1.326071,1.326071,0.139983,0.087685,19.800723,13.605240,8.804037,4.214494,1.000000,18.361833
4,0,1,0,0,0,0,0,1,0,0,...,0.000000,0.000000,0.000000,0.000000,18.767298,18.767298,0.000000,0.000000,3.000000,59.110850
5,0,1,0,0,0,1,0,0,0,0,...,1.708773,1.708773,1.059802,0.053767,28.728905,16.445445,8.294372,3.533376,1.000000,20.664418


In [215]:
CF.Calnan(data_complete)["nancount"].value_counts()  # 可见，全部填充完毕，没有空值。

0    424
Name: nancount, dtype: int64

## 5.5 整合总表

In [216]:
# 调整一下顺序，把loan_time放在倒数第二，然后倒数第一列是添加的目标列 overdue（即target）。
# 得到的 data1 就是填充好的数据。里面除了 target的未知样本集部分是空的，其它都是无缺失的。

# data1 就是填充了缺失值的整个数据集（除target列外，其它全部填充）。（包括训练集+未知样本集）。以 userID为行索引。
# 其 shape 为：(69280, 425)
# 其中，user_info_OHE 为 24列，bank_detail_OHE 为 24列，browse_history_stats 为 76列，bill_detail_stats 为 299 列，
# loan_time 1列，target 1列。
# 其中，训练集 55381行，未知样本集 13899行。

data1 = pd.concat([data_complete.drop(["loan_time"],axis=1),pd.DataFrame(data_origin_2["loan_time"]),pd.DataFrame(data_origin_2["target"])],axis=1)
print("shape of data1 is:",data1.shape)
data1.head()

shape of data1 is: (69280, 425)


,gender_OHE_0,gender_OHE_1,gender_OHE_2,career_OHE_0,career_OHE_1,career_OHE_2,career_OHE_3,career_OHE_4,education_OHE_0,education_OHE_1,...,bill_AL_cardM__available_balance_median,bill_AL_cardM__available_balance_min,bill_AL_cardM__borrow_cash_sum,bill_AL_cardM__borrow_cash_max,bill_AL_cardM__borrow_cash_median,bill_AL_cardM__borrow_cash_min,bill_card_count,bill_credit_limit_cardsum,loan_time,target
userID,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,1,0,0,0,0,...,0.644378,0.049019,25.766646,14.131988,11.047256,5.839872,2.897174,44.916014,5914855887,0.0
2,0,1,0,0,0,1,0,0,0,0,...,0.000000,0.000000,20.376736,20.376736,0.000000,0.000000,3.000000,61.523250,5914855887,0.0
3,0,1,0,0,0,0,0,1,0,0,...,0.139983,0.087685,19.800723,13.605240,8.804037,4.214494,1.000000,18.361833,5914855887,0.0
4,0,1,0,0,0,0,0,1,0,0,...,0.000000,0.000000,18.767298,18.767298,0.000000,0.000000,3.000000,59.110850,5914855887,1.0
5,0,1,0,0,0,1,0,0,0,0,...,1.059802,0.053767,28.728905,16.445445,8.294372,3.533376,1.000000,20.664418,5914855887,0.0


In [217]:
CF.Calnan(data1.loc[:,"gender_OHE_0":"loan_time"])["nancount"].value_counts()  # 可见，除target外，全部填充完毕，没有空值。

0    424
Name: nancount, dtype: int64

In [218]:
CF.Calnan(pd.DataFrame(data1.loc[:,"target"]))   # target列有空值。

,column_name,nancount,nanpercent
0,target,13899,0.200621


In [219]:
# 把 已经填充过缺失值的 整个数据表（带target列）存成名为 data1 的 csv 文件，注意，这是 训练集+未知样本集。

data1.reset_index().to_csv(r'F:\RiskPre2\Data clean 1\data1.csv',index=False)

In [220]:
# 把 训练集和未知样本集的 userID 分别存储为 userIDtrain 和 userIDtest 两个文件，为日后分隔训练集和未知样本集用。

TR = pd.DataFrame(userIDtrain,columns=["userIDtrain"])
TE = pd.DataFrame(userIDtest,columns=["userIDtest"])
TR.to_csv(r'F:\RiskPre2\Data clean 1\userIDtrain.csv',index=False)
TE.to_csv(r'F:\RiskPre2\Data clean 1\userIDtest.csv',index=False)

In [221]:
TR.shape

(55381, 1)

In [222]:
d = pd.read_csv(r'F:\RiskPre2\Data clean 1\userIDtest.csv')